In [1]:
import pymongo, os
import pandas as pd
import re

In [2]:
client = pymongo.MongoClient('localhost', 27017)

In [3]:
db = client["WATSON"]

In [4]:
DESKTOP = os.path.join(os.environ["HOME"], "Desktop")
base_dir = os.path.join(DESKTOP, "trial_stats")
targeted_therapy_trials = [ line.strip() for line in open(os.path.join(base_dir, "targeted_therapy_ncts.txt")) if line != "" ]
existing_trials = [ line.strip() for line in open(os.path.join(base_dir, "existing_trials_wfg.txt")) if line != "" ]
cancer_genes = pd.read_csv(os.path.join(base_dir, "cancer_genes50.txt"),header=None)[0]
cancer_genes = list(cancer_genes)
new_targeted_therapy_trials = set(targeted_therapy_trials).difference(set(existing_trials))

In [5]:
studies = db["CTGOV"].find({"NCT_ID" : {"$in" : targeted_therapy_trials}})

In [6]:
df = pd.DataFrame(list(studies))

In [7]:
all_countries = []
for countries in df["COUNTRIES"]:
    all_countries.extend(countries)
    
all_countries = set(all_countries)

In [11]:
len(df["NCT_ID"])

4681

In [17]:
df.pivot(index='STATUS', columns='NCT_ID', values='COUNTRIES')

NCT_ID,NCT00001337,NCT00001379,NCT00003553,NCT00006436,NCT00026078,NCT00073957,NCT00074282,NCT00075387,NCT00091234,NCT00104676,...,NCT02882984,NCT02883556,NCT02884648,NCT02885324,NCT02885727,NCT02885753,NCT02885974,NCT02886000,NCT02886065,NCT02887040
STATUS,,,,,,,,,,,,,,,,,,,,,
"Active, not recruiting",None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Completed,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Enrolling by invitation,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Not yet recruiting,None,None,None,None,None,None,None,None,None,None,...,None,[],[United States],[United States],[],[],[United States],None,[United States],[]
Recruiting,[United States],[United States],[United States],[United States],"[Argentina, Mexico]",[United States],[United States],[United States],"[Belgium, Italy, Netherlands]","[France, Slovakia, United States]",...,[China],None,None,None,None,None,None,"[Puerto Rico, United States]",None,None
Suspended,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Terminated,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Withdrawn,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [36]:
grouped=df.groupby(['STATUS'])

In [37]:
print grouped.size()

STATUS
Active, not recruiting      312
Completed                    33
Enrolling by invitation       5
Not yet recruiting          594
Recruiting                 3643
Suspended                    34
Terminated                   41
Withdrawn                    19
dtype: int64


In [16]:
drug_db = pd.read_csv(os.path.join(base_dir, "WGA_ALL_DRUG_GENERAL.tsv"), delimiter="\t", header=None)

In [85]:
#wga_db_ids = list(drug_db[0])
((drug_db[0] == "DB00888") & (drug_db[9] == "N")).any()

True

In [9]:
new_targeted_therapy_trials = set(targeted_therapy_trials).difference(set(existing_trials))

In [10]:
db_genes = []
for record in db.GENE.find({"SYMBOL": {"$in" : cancer_genes}}):
    db_genes += [ alias.strip() for alias in record["ALIAS"] if alias.strip() != ""]

In [11]:
GENE_REGEX = re.compile("|".join(["\\b"+gene+"\\b" for gene in cancer_genes + db_genes]))
text = "PATIENT CHARACTERISTICS:"
print GENE_REGEX.findall(text)

[]


In [17]:
new_studies = db["CTGOV"].find({"NCT_ID" : {"$in" : list(new_targeted_therapy_trials)}, "COUNTRIES" : "United States"})
print new_studies.count()

trials_of_interest = pd.DataFrame()
column_genes = []
column_drugs = []
column_trial = []
for study in new_studies:
    temp_drugs = [ intervention["NAME"] for intervention in study["interventions"] if ((drug_db[0] == intervention.get("WGADB_ID", "")) & (drug_db[9] == "Y")).any() ]
    temp_genes = set(GENE_REGEX.findall(study["inclusion"]["annotation"].get("text", "") + "\n" + study["exclusion"]["annotation"].get("text", "")))
    column_drugs.append(", ".join(temp_drugs))
    column_genes.append(", ".join(temp_genes))
    column_trial.append(study["NCT_ID"])
    
trials_of_interest["NCT_ID"] = column_trial
trials_of_interest["GENES"] = column_genes
trials_of_interest['DRUGS'] = column_drugs

1398


In [18]:
trials_of_interest.to_csv(os.path.join(base_dir,"trials_of_interest.csv"))

In [141]:
quest_drugs = pd.read_csv(os.path.join(base_dir, "Drugs_for_Quest_Panel.tsv"), delimiter="\t")
print quest_drugs["Therapy ID"]
new_studies = db["CTGOV"].find({"NCT_ID" : {"$in" : list(new_targeted_therapy_trials)}, "COUNTRIES" : "United States"})
all_drug_to_trial_dicts = []
for study in new_studies:
    wga_drug_ids = [ intervention.get("WGADB_ID") for intervention in study["interventions"] if "WGADB_ID" in intervention ]
    temp = [ (wga_drug_id, study["NCT_ID"]) for wga_drug_id in wga_drug_ids ]
    all_drug_to_trial_dicts.append(temp)
    


0         5088
1          570
2         4019
3         4029
4         3303
5         5102
6         5060
7         1059
8         1037
9         5199
10     DB05773
11     DB08875
12        5045
13     DB00002
14        4128
15         359
16        5017
17     DB08877
18        5065
19        5014
20        4104
21        5188
22        5058
23         648
24         820
25         263
26        5165
27        4083
28         335
29         129
        ...   
243       3058
244       5193
245    DB06589
246       4080
247       3026
248       4115
249       1000
250        649
251       4428
252       5215
253       5084
254        726
255       5220
256    DB06626
257    DB02546
258       4097
259       5194
260       5238
261       5159
262       5206
263       5039
264        156
265       5126
266       1069
267    DB01268
268        549
269       1281
270       4017
271       4058
272       1197
Name: Therapy ID, dtype: object
